<a href="https://colab.research.google.com/github/hoa92ng/Homework/blob/main/Welcome_To_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
test

In [ ]:
from torchaudio.datasets import SPEECHCOMMANDS

In [ ]:
label_2_id = {'yes':0,
              'no':1,
              'up':2,
              'down':3,
              'left':4,
              'right':5,
              'on':6,
              'off':7,
              'stop':8,
              'go':9,
              'silence':10,
              'unknown':11, }
id_2_label = {0:'yes',
              1:'no',
              2:'up',
              3:'down',
              4:'left',
              5:'right',
              6:'on',
              7:'off',
              8:'stop',
              9:'go',
              10:'silence',
              11:'unknown',}

In [ ]:
import torchaudio
from torchaudio.datasets import SPEECHCOMMANDS
from torch.utils.data import DataLoader, Dataset
import os

# Define a custom dataset class to load Google Speech Commands
class GoogleSpeechCommands(Dataset):
    def __init__(self, subset=None):
        self.dataset = SPEECHCOMMANDS(root="./data", url='speech_commands_v0.01', download=True, subset=subset)

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        waveform, sample_rate, label, _, _ = self.dataset[idx]

        return waveform, sample_rate, label

# Create train and test datasets
train_dataset = GoogleSpeechCommands(subset='training')
test_dataset = GoogleSpeechCommands(subset='validation')
testt_dataset = GoogleSpeechCommands(subset='testing')

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
waveform, sample_rate, label, _, _ = test_dataset.dataset[3351]
print(waveform, sample_rate, label, _, _ )
if label in label_2_id.keys():
    print(label_2_id[label])
else:
    print(label_2_id['unknown'])

In [ ]:
import torch
import torch.nn as nn
from transformers import Wav2Vec2Model

# Define the Wav2Vec2 model with a classification head
class Wav2Vec2ForSpeechCommandClassification(nn.Module):
    def __init__(self, num_classes, freeze_wav2vec=True):
        super(Wav2Vec2ForSpeechCommandClassification, self).__init__()
        self.wav2vec = Wav2Vec2Model.from_pretrained("./model_1")

        # Freeze Wav2Vec2 parameters if specified
        if freeze_wav2vec:
            for param in self.wav2vec.parameters():
                param.requires_grad = False

        self.classifier = nn.Linear(self.wav2vec.config.hidden_size, num_classes)

    def forward(self, input_values):
        with torch.no_grad():  # Skip gradient calculation for frozen layers
            features = self.wav2vec(input_values).last_hidden_state
        features = features.mean(dim=1)
        logits = self.classifier(features)
        return logits

# Number of commands in the dataset
num_classes = 35  # Adjust based on the number of commands in your dataset

model = Wav2Vec2ForSpeechCommandClassification(num_classes)

In [ ]:
from torch.optim import AdamW
import torch.nn.functional as F

# Define optimizer and loss function
optimizer = AdamW(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def train(model, train_loader, optimizer, criterion):
    model.train()
    total_loss = 0
    for batch in train_loader:
        inputs, sample_rate, labels = batch
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        logits = model(inputs)
        loss = criterion(logits, labels)
        loss.backward()

        optimizer.step()
        total_loss += loss.item()

    return total_loss / len(train_loader)

def evaluate(model, test_loader, criterion):
    model.eval()
    total_loss = 0
    correct = 0
    with torch.no_grad():
        for batch in test_loader:
            inputs, sample_rate, labels = batch
            inputs = inputs.to(device)
            labels = labels.to(device)

            logits = model(inputs)
            loss = criterion(logits, labels)
            total_loss += loss.item()

            preds = torch.argmax(logits, dim=-1)
            correct += torch.sum(preds == labels).item()

    accuracy = correct / len(test_loader.dataset)
    return total_loss / len(test_loader), accuracy

In [ ]:
num_epochs = 10

for epoch in range(num_epochs):
    train_loss = train(model, train_loader, optimizer, criterion)
    test_loss, accuracy = evaluate(model, test_loader, criterion)

    print(f"Epoch {epoch + 1}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Accuracy: {accuracy:.4f}")

In [ ]:
torch.save(model.state_dict(), "wav2vec2_speech_commands.pth")